In [1]:
options(jupyter.rich_display=FALSE) # Create output as usual in R

In order to run this notebook successfully, you have to download the 1983 British Election Study from the [BES website](https://www.britishelectionstudy.com/data-object/1983-bes-cross-section/) and upload it to the virtual machine on which this notebook runs. To do this, 

1. pull down the "File" menu item and select "Open"
2. An overview of the folder that contains the notebook opens. 
3. The folder view has a button labelled "Upload". Use this to upload the file that you downloaded from the BES website. Its name should be `83BES.sav`.

Note that the uploaded data will disappear, once you "Quit" the notebook (and the Jupyter instance).

In [2]:
library(memisc) # The functions used here are in this package.

Loading required package: lattice

Loading required package: MASS


Attaching package: ‘memisc’


The following objects are masked from ‘package:stats’:

    contr.sum, contr.treatment, contrasts


The following object is masked from ‘package:base’:

    as.array




In [3]:
# The first step: Optainin the location and description of the survey data file
BES.1983.por <- spss.portable.file("83BES.sav")

In [4]:
description(BES.1983.por[1:30])


 ID1      'SERIAL NUMBER'                           
 RECORD1  'CARD NUMBER'                             
 AREANO   'AREA NUMBER'                             
 PANO     'PA CONSTITUENCY NUMBER'                  
 MOVER    'MOVER-REISSUE CODE'                      
 INT      'INTERVIEWER NUMBER'                      
 FGLCLASS 'FATHER'S GOLDTHORPE-LLEWELLYN CLASS'     
 RGLCLASS 'RESPONDENT'S GOLDTHORPE-LLEWELLYN CLASS' 
 SGLCLASS 'SPOUSE'S GOLDTHORPE-LLEWELLYN CLASS'     
 FGHCLASS 'FATHER'S GOLDTHORPE-HEATH CLASS'         
 RGHCLASS 'RESPONDENT'S GOLDTHORPE-HEATH CLASS'     
 SGHCLASS 'SPOUSE'S GOLDTHORPE-HEATH CLASS'         
 Q1       'CARE WHICH PARTY WON'                    
 Q2A      'FOLLOWED ELECTION BROADCASTS'            
 Q2B      'READ ABOUT CAMPAIGN IN NEWSPAPER'        
 Q2C      '[IF READ NEWSPP] WHICH DAILY ON CAMPAIGN'
 Q2D      'HEARD CANDIDATE AT POLITICAL MEETING'    
 Q2E      'CANVASSED FOR CANDIDATE'                 
 Q2F      'PUT UP POLITICAL PARTY POSTER'    

In [5]:
# The second step: Loading a subset of the data - only the variables that are
# really needed for the analysis.
BES.1983.classvot <- subset(
    BES.1983.por,
    select=c(
        voted          = Q7A,
        vote           = Q9A,
        lrself         = Q46G,
        fglclass       = FGLCLASS,
        rglclass       = RGLCLASS,
        sglclass       = SGLCLASS,
        ethnicity      = Q64A,
        gender         = Q64B,
        age            = Q56,
        educ           = Q59A,
        religion       = Q63A,
        religatt       = Q63B
    ))
description(BES.1983.classvot)


 voted     'DID RESPONDENT VOTE 1983'               
 vote      '[IF VOTED] PARTY VOTED FOR'             
 lrself    'LEFT,RIGHT SCALE: R'S POSITION 1983'    
 fglclass  'FATHER'S GOLDTHORPE-LLEWELLYN CLASS'    
 rglclass  'RESPONDENT'S GOLDTHORPE-LLEWELLYN CLASS'
 sglclass  'SPOUSE'S GOLDTHORPE-LLEWELLYN CLASS'    
 ethnicity 'RESPONDENT'S ETHNIC ORIGIN'             
 gender    'RESPONDENT'S SEX'                       
 age       'RESPONDENT'S AGE LAST BIRTHDAY'         
 educ      'AGE RESPONDENT LEFT SCHOOL'             
 religion  'RESPONDENT'S RELIGION'                  
 religatt  '[IF RELIGION] FREQUENCY OF ATTENDANCE'  


In [6]:
# Declaring the measurement level of a single variable
measurement(BES.1983.classvot$lrself) <- "interval"

In [7]:
# Declaring the measurement level of several variables
BES.1983.classvot <- within(BES.1983.classvot,{
   measurement(lrself) <- "interval"
   measurement(age) <- "interval"
   measurement(educ) <- "interval"
})

In [8]:
# Declaring the measurement level of several variables using a loop
BES.1983.classvot <- within(BES.1983.classvot,{
   foreach(var=c(lrself,age,educ),{
           measurement(var) <- "interval"
   })
})

In [9]:
# Obtaining the codebook a single variable
codebook(BES.1983.classvot["age"])


   age 'RESPONDENT'S AGE LAST BIRTHDAY'

--------------------------------------------------------------------------------

   Storage mode: double
   Measurement: interval

   Values and labels       N Percent
                                    
   99 'NA'                20     0.5
                                    
        Min: 18.000                 
        Max: 99.000                 
       Mean: 45.704                 
   Std.Dev.: 18.174                 


In [10]:
# Declaring the missing values for this variable
missing.values(BES.1983.classvot$age) <- 99

In [11]:
# Declaring several missing values
BES.1983.classvot <- within(BES.1983.classvot,{
    missing.values(voted)     <- 9
    missing.values(vote)      <- 95:99
    missing.values(lrself)    <- 95:99
    missing.values(fglclass)  <- 0
    missing.values(rglclass)  <- 0
    missing.values(sglclass)  <- 0
    missing.values(ethnicity) <- 9
    missing.values(gender)    <- 9
    missing.values(age)       <- 9
    missing.values(age)       <- 98:99
    missing.values(religion)  <- 99
    missing.values(religatt)  <- 7:9
})

In [12]:
# The codebook of the resulting data set object
codebook(BES.1983.classvot)


   voted 'DID RESPONDENT VOTE 1983'

--------------------------------------------------------------------------------

   Storage mode: double
   Measurement: nominal
   Missing values: 9

   Values and labels       N Percent
                                    
   1   'YES'            3295    83.3
   2   'NO'              660    16.7


   vote '[IF VOTED] PARTY VOTED FOR'

--------------------------------------------------------------------------------

   Storage mode: double
   Measurement: nominal
   Missing values: 95, 96, 97, 98, 99

   Values and labels         N Valid Total
                                          
    0   'SKIPPED'          660  17.1  16.7
    1   'CON'             1432  37.1  36.2
    2   'LAB'              937  24.3  23.7
    3   'ALLIANCE'         416  10.8  10.5
    4   'LIB'              278   7.2   7.0
    5   'SOCIAL  DEMOCR'    94   2.4   2.4
    6   'SNP'               33   0.9   0.8
    7   'PLAID   CYMRU'      7   0.2   0.2
    8   'ECOLOGY PARTY'

In [13]:
options(jupyter.rich_display=TRUE)
show_html(codebook(BES.1983.classvot)) # Same information in HTML format

<div>
<div class="codebook">
<div class="codebook-entry">
<div class="cbe-header" style="border-top: 1px solid; border-bottom: 1px solid; padding-left: 3ex;">
<p></p>
<p class="cbe-title" id="voted"><code class="cbe-name">voted</code> &mdash; <span class="cbe-description">'DID RESPONDENT VOTE 1983'</span></p></div>
<div class="cbe-body" style="padding-left: 3ex;">
<p></p>
<table class="cbe-spec">
<tr><td style="text-align: left;">Storage mode:</td><td style="text-align: left;">double</td></tr>
<tr><td style="text-align: left;">Measurement:</td><td style="text-align: left;">nominal</td></tr>
<tr><td style="text-align: left;">Missing values:</td><td style="text-align: left;">9</td></tr>
</table>
<p></p>
<table class="cbe-table" style="border-collapse: collapse;">
<tr><td colspan="3" style="text-align: center;">Values and labels</td><td style="text-align: center;">N</td><td colspan="3" style="text-align: center;">Percent</td></tr>
<tr><td style="text-align: right; padding-left: 0.3em; padding-right: 0.3em;">1</td><td style="text-align: center; padding-left: 0.3em; padding-right: 0.3em;"></td><td style="text-align: left; padding-left: 0.3em; padding-right: 0.3em;">'YES'</td><td style="text-align: right; padding-left: 0.3em; padding-right: 0.3em;">3295</td><td style="text-align: right; margin-right: 0px; padding-right: 0px; padding-left: 0.3em;">83</td><td style="text-align: center; margin-left: 0px; margin-right: 0px; padding-right: 0px; padding-left: 0px; width: 1px;">.</td><td style="text-align: left; margin-left: 0px; padding-left: 0px; padding-right: 0.3em;">3</td></tr>
<tr><td style="text-align: right; padding-left: 0.3em; padding-right: 0.3em;">2</td><td style="text-align: center; padding-left: 0.3em; padding-right: 0.3em;"></td><td style="text-align: left; padding-left: 0.3em; padding-right: 0.3em;">'NO'</td><td style="text-align: right; padding-left: 0.3em; padding-right: 0.3em;">660</td><td style="text-align: right; margin-right: 0px; padding-right: 0px; padding-left: 0.3em;">16</td><td style="text-align: center; margin-left: 0px; margin-right: 0px; padding-right: 0px; padding-left: 0px; width: 1px;">.</td><td style="text-align: left; margin-left: 0px; padding-left: 0px; padding-right: 0.3em;">7</td></tr>
</table>
<p></p>
</div>
</div>
<div class="codebook-entry">
<div class="cbe-header" style="border-top: 1px solid; border-bottom: 1px solid; padding-left: 3ex;">
<p></p>
<p class="cbe-title" id="vote"><code class="cbe-name">vote</code> &mdash; <span class="cbe-description">'[IF VOTED] PARTY VOTED FOR'</span></p></div>
<div class="cbe-body" style="padding-left: 3ex;">
<p></p>
<table class="cbe-spec">
<tr><td style="text-align: left;">Storage mode:</td><td style="text-align: left;">double</td></tr>
<tr><td style="text-align: left;">Measurement:</td><td style="text-align: left;">nominal</td></tr>
<tr><td style="text-align: left;">Missing values:</td><td style="text-align: left;">95, 96, 97, 98, 99</td></tr>
</table>
<p></p>
<table class="cbe-table" style="border-collapse: collapse;">
<tr><td colspan="3" style="text-align: center;">Values and labels</td><td style="text-align: center;">N</td><td colspan="3" style="text-align: center;">Valid</td><td colspan="3" style="text-align: center;">Total</td></tr>
<tr><td style="text-align: right; padding-left: 0.3em; padding-right: 0.3em;">0</td><td style="text-align: center; padding-left: 0.3em; padding-right: 0.3em;"></td><td style="text-align: left; padding-left: 0.3em; padding-right: 0.3em;">'SKIPPED'</td><td style="text-align: right; padding-left: 0.3em; padding-right: 0.3em;">660</td><td style="text-align: right; margin-right: 0px; padding-right: 0px; padding-left: 0.3em;">17</td><td style="text-align: center; margin-left: 0px; margin-right: 0px; padding-right: 0px; padding-left: 0px; width: 1px;">.</td><td style="text-align: left; margin-left: 0px; padding-left: 0px; padding-right: 0.3em;">1</td><td style="text-align: right; margin-right: 0px; padding-right: 0px; padding-left: 0.

In [14]:
save(BES.1983.classvot,
     file="BES-1983-classvot.RData")